In [1]:
from folktables import ACSDataSource
import folktables
import pandas as pd
import json
import time
import numpy as np
import random
import pickle
from folktables.load_acs import state_list

In [2]:
!pwd

/home/jupyter-kriach/folktables/multigroup-code


# Schema for downloading the data

In [3]:
print(state_list) # all states

['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR']


In [ ]:
# state_list = ['GA','PA', 'NJ'] # Georgia, Pennsylvania, New Jersey
# try this out if all states don't fit in machine memory

In [4]:
ACSIncome = folktables.BasicProblem(
    features=[
#         'ST',
        'AGEP', #Numeric, age of householder
        'COW',  #Categorical, class of worker 
        'SCHL', #Categorical, educational attainment
        'MAR', #Categorical, Mamarital status 5 categories 
        'OCCP', #Categorical, occupation lots of codes here
#         'POBP', #place of birth, US states, and if international has places
#         'RELSHIPP', #Relationship, renamed to Relantioship in the new PUMS syntax, 20 - 38
        'WKHP', #Numeric, hours worked per week in last 12 months
        'SEX',  #Categorical Male, Female 2 
        'RAC1P', #Categorical Recoded detailed race code, 9 categories here 1)White ... 9
    ],
    target='PINCP',
    group = 'RAC1P', #ignore this flag
    preprocess=folktables.adult_filter, # age of householder > 16, etc.., see acs.py in folktables
    postprocess=lambda x: np.nan_to_num(x, -1),
) #default ACSIncome Pull above


In [5]:
# random.seed(42) not shuffling, state wise round robin, kind of non-stochastic setting
# random.shuffle(state_list)
print('Selected states: ', state_list)
data_source = ACSDataSource(survey_year='2021', horizon='1-Year', survey='person')
acs_data = data_source.get_data(state_list, download=True)

Selected states:  ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR']


In [6]:
X, y, gr = ACSIncome.df_to_pandas(acs_data)
dataset = pd.concat([X, y], axis=1)

In [7]:
dataset

,AGEP,COW,SCHL,MAR,OCCP,WKHP,SEX,RAC1P,PINCP
0,19.0,2.0,18.0,5.0,4760.0,30.0,2.0,1.0,8000.0
1,20.0,1.0,19.0,5.0,4640.0,40.0,1.0,2.0,6300.0
2,19.0,2.0,18.0,5.0,5240.0,18.0,2.0,1.0,6200.0
3,34.0,2.0,19.0,3.0,4220.0,6.0,2.0,1.0,10800.0
4,19.0,1.0,18.0,5.0,2722.0,10.0,1.0,1.0,2000.0
...,...,...,...,...,...,...,...,...,...
1630162,40.0,1.0,21.0,5.0,1430.0,40.0,2.0,9.0,80000.0
1630163,47.0,2.0,22.0,1.0,2205.0,40.0,1.0,9.0,60000.0
1630164,46.0,4.0,21.0,1.0,5740.0,40.0,2.0,9.0,18600.0
1630165,22.0,6.0,21.0,5.0,2634.0,30.0,1.0,9.0,12200.0


# Code blocks below process the dataframe

2 filtering operations

    -We end up removing those individuals earning more than 200k
    
    -and only pick the individuals in the top 200 occupation code buckets

2 scaling operations

    -minmax scaling for numeric features
    
    -one hot encoding for categorical

In [8]:
#important to explicity define columns with categoricals as ints to index properly into them
dataset['COW'] = dataset['COW'].astype(int)
dataset['SCHL'] = dataset['SCHL'].astype(int)
dataset['MAR'] = dataset['MAR'].astype(int)
dataset['OCCP'] = dataset['OCCP'].astype(int)
dataset['SEX'] = dataset['SEX'].astype(int)
dataset['RAC1P'] = dataset['RAC1P'].astype(int)

In [9]:
pd.set_option('display.float_format', lambda x: '%.2f' % x) # formatting for only 2 decimal places
dataset.describe()

,AGEP,COW,SCHL,MAR,OCCP,WKHP,SEX,RAC1P,PINCP
count,1630167.00,1630167.00,1630167.00,1630167.00,1630167.00,1630167.00,1630167.00,1630167.00,1630167.00
mean,43.62,2.12,18.69,2.56,4068.13,38.07,1.48,2.72,63057.14
std,15.41,1.84,3.44,1.81,2715.57,12.94,0.50,2.94,79616.10
min,17.00,1.00,1.00,1.00,10.00,1.00,1.00,1.00,110.00
25%,31.00,1.00,16.00,1.00,2014.00,35.00,1.00,1.00,22000.00
50%,43.00,1.00,19.00,1.00,4030.00,40.00,1.00,1.00,43000.00
75%,56.00,3.00,21.00,5.00,5740.00,40.00,2.00,2.00,75000.00
max,96.00,8.00,24.00,5.00,9830.00,99.00,2.00,9.00,1604700.00


In [10]:
dataset['PINCP'].describe(percentiles=[.3, .6, .95]) # so 95 % make less than 200k, lets filter richer out

count   1630167.00
mean      63057.14
std       79616.10
min         110.00
30%       26000.00
50%       43000.00
60%       53000.00
95%      178000.00
max     1604700.00
Name: PINCP, dtype: float64

In [11]:
data_incomeless200k = dataset[dataset['PINCP'] <= 2e5]

In [12]:
data_incomeless200k

,AGEP,COW,SCHL,MAR,OCCP,WKHP,SEX,RAC1P,PINCP
0,19.00,2,18,5,4760,30.00,2,1,8000.00
1,20.00,1,19,5,4640,40.00,1,2,6300.00
2,19.00,2,18,5,5240,18.00,2,1,6200.00
3,34.00,2,19,3,4220,6.00,2,1,10800.00
4,19.00,1,18,5,2722,10.00,1,1,2000.00
...,...,...,...,...,...,...,...,...,...
1630162,40.00,1,21,5,1430,40.00,2,9,80000.00
1630163,47.00,2,22,1,2205,40.00,1,9,60000.00
1630164,46.00,4,21,1,5740,40.00,2,9,18600.00
1630165,22.00,6,21,5,2634,30.00,1,9,12200.00


In [13]:
top_200OCCP = dataset['OCCP'].value_counts()[:250].index.tolist() # top 200 occupation codes in the original dataframe
data_fil = data_incomeless200k[data_incomeless200k['OCCP'].isin(top_200OCCP)]
print(len(data_fil) / len(dataset)) #top200 occp codes and <= 200k is 83 % of original dataset

0.8761574734367706


In [14]:
data_fil

,AGEP,COW,SCHL,MAR,OCCP,WKHP,SEX,RAC1P,PINCP
0,19.00,2,18,5,4760,30.00,2,1,8000.00
1,20.00,1,19,5,4640,40.00,1,2,6300.00
2,19.00,2,18,5,5240,18.00,2,1,6200.00
3,34.00,2,19,3,4220,6.00,2,1,10800.00
4,19.00,1,18,5,2722,10.00,1,1,2000.00
...,...,...,...,...,...,...,...,...,...
1630162,40.00,1,21,5,1430,40.00,2,9,80000.00
1630163,47.00,2,22,1,2205,40.00,1,9,60000.00
1630164,46.00,4,21,1,5740,40.00,2,9,18600.00
1630165,22.00,6,21,5,2634,30.00,1,9,12200.00


In [15]:
from sklearn.preprocessing import MinMaxScaler
def numeric_scaler(df, numeric_cols):
    '''
    df: pandas dataframe
    numeric_cols: (array of strings) column names for numeric variables

    no return: does inplace operation
    '''
    numeric_scaler = MinMaxScaler()
    df[numeric_cols] = numeric_scaler.fit_transform(df[numeric_cols])

def one_hot(df, catergoric_cols):
    """
    df: pandas DataFrame
    param: cols a list of columns to encode 
    return a DataFrame with one-hot encoding
    """
    for each in catergoric_cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df

In [16]:
numeric_scaler(data_fil, ['AGEP', 'WKHP', 'PINCP']) #inplace scaling of numeric columns in data_fil dataframe

/tmp/ipykernel_1741308/1616867230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_cols] = numeric_scaler.fit_transform(df[numeric_cols])


In [17]:
categorical = ['COW','SCHL', 'MAR', 'OCCP', 'SEX', 'RAC1P'] # categorical variables need to be one-hot encoded
data_fil_oh = one_hot(data_fil, categorical)
df_final = data_fil_oh.drop(columns = categorical)

In [18]:
df_final # final dataframe which is properly scaled and encoded, can be fed to downstream linear model regression tasks

,AGEP,WKHP,PINCP,COW_1,COW_2,COW_3,COW_4,COW_5,COW_6,COW_7,...,SEX_2,RAC1P_1,RAC1P_2,RAC1P_3,RAC1P_4,RAC1P_5,RAC1P_6,RAC1P_7,RAC1P_8,RAC1P_9
0,0.03,0.30,0.04,0,1,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,0.04,0.40,0.03,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0.03,0.17,0.03,0,1,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,0.22,0.05,0.05,0,1,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
4,0.03,0.09,0.01,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630162,0.29,0.40,0.40,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1630163,0.38,0.40,0.30,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1630164,0.37,0.40,0.09,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1630165,0.06,0.30,0.06,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
#saving the pandas dataframe to a pickle, uncomment below
df_final.to_pickle("./allstates2021_roundrobin.pkl")